# Patient Data Analysis

In [149]:
# import libraries
import pandas as pd
from pathlib import Path
import numpy as np
import os
from dateutil.relativedelta import relativedelta


import sys
sys.path.append(r"/Users/LennartPhilipp/Desktop/Uni/Prowiss/Code/Brain_Mets_Classification")
import brain_mets_classification.config as config

### load data

load excel sheet

In [191]:
# reads the patient excel sheet and puts them into a pandas Dataframe
path_to_excel = Path("/Users/LennartPhilipp/Desktop/Uni/Prowiss/Dateien/patient data analysis.xlsx")
try:
    patientsDataFrame = pd.read_excel(path_to_excel)
except ImportError as e:
    print(e)

modifiedPatientsDataFrame = patientsDataFrame.copy()

load patients actually in dataset

In [192]:
path_to_list_of_patients_in_dataset = Path("/Users/LennartPhilipp/Desktop/Uni/Prowiss/Dateien/dataset_patients_05_24.tsv")

dataset_patients = pd.read_csv(path_to_list_of_patients_in_dataset, sep="\t", index_col=False)

In [3]:
path_to_dataset_patients = Path("/Users/LennartPhilipp/Desktop/Uni/Prowiss/Dateien/participants.tsv")
path_to_preprocessed_images = Path("/Volumes/BrainMets/Rgb_Brain_Mets/brain_mets_classification/derivatives/preprocessed_brainlesion_allpatients")

dataset_patients = pd.read_csv(path_to_dataset_patients, sep="\t", index_col=False)

# drop patient sub-01383503
patient_to_drop_index = dataset_patients.index[dataset_patients["participant_id"] == "sub-01383503"]
dataset_patients.drop(index=patient_to_drop_index, inplace=True)
dataset_patients.reset_index(drop=True, inplace=True)
#training_patients = training_patients.sample(frac=1) #shuffles the dataset

print(f"Before removing unfit files: {len(dataset_patients)}")
patient_files_list = os.listdir(path_to_preprocessed_images)
for index, row in dataset_patients.iterrows():
    if dataset_patients["participant_id"][index] not in patient_files_list:
        dataset_patients.drop(index=index, inplace=True)

print(f"After removing unfit files: {len(dataset_patients)}")
dataset_patients.reset_index(drop=True, inplace=True)

Before removing unfit files: 493
After removing unfit files: 491


In [4]:
# export dataset_patients
path_to_list_of_patients_in_dataset = Path("/Users/LennartPhilipp/Desktop/Uni/Prowiss/Dateien/dataset_patients_05_24.tsv")
dataset_patients.to_csv(path_to_list_of_patients_in_dataset, index=False, sep="\t")

### Data clean up

In [193]:
# removes unused columns from dataset
modifiedPatientsDataFrame = modifiedPatientsDataFrame.iloc[:, :39] # only keeps the columns until column 38
modifiedPatientsDataFrame = modifiedPatientsDataFrame.drop(columns=["T.1",
                                                                    "primary_coded",
                                                                    "primary_coded_clean",
                                                                    "N",
                                                                    "M",
                                                                    "M_number_organ_systems",
                                                                    "TNM",
                                                                    "Survival ab ED gesamt",
                                                                    "Survival ab ED Hirnmet.",
                                                                    "Survival_from_ED_Met (months)",
                                                                    "Alter bei MRT (J./kalkuliert)",
                                                                    "Dauer Tumorleiden\n(kalkuliert; von ED bis ED Hirnmet.)",
                                                                    "Karnofsky (%)",
                                                                    "periphere Metastasen zum Zeitpunkt des MRTs ( Keine = 0, Einzelne = 1, Oligo =2, Multipel =3)",
                                                                    "im Verlauf neu aufgetretene Metastasen (0 = keine bekannt)",
                                                                    ])

# turn ID strings into ints
modifiedPatientsDataFrame.ID = modifiedPatientsDataFrame.ID.astype(int)

dataset_patients["participant_id"] = dataset_patients["participant_id"].str.slice_replace(0, 4, "").astype(int)

### patients selected by whom

In [194]:
print(modifiedPatientsDataFrame["selected by"].value_counts())

selected by
Lennart        483
not Lennart    262
Name: count, dtype: int64


#### patients not selected by Lennart
to do:
- put "MRI sequences missing" into the "reason" column in excel based on unused_pats_not_Lennart

In [195]:
# compare patients not selected by Lennart with dataset_patients
pats_not_used_not_Lennart = []
#pats_not_by_Lennart = modifiedPatientsDataFrame.query("`selected by` == not Lennart")["ID"]
pats_not_by_Lennart = list(modifiedPatientsDataFrame[modifiedPatientsDataFrame["selected by"] == "not Lennart"]["ID"])

print(f"# of patients not selected by Lennart: {len(pats_not_by_Lennart)}")

amount_of_patients_used_not_lennart = 0

for pat_id in pats_not_by_Lennart:
    if pat_id not in list(dataset_patients["participant_id"]):
        pats_not_used_not_Lennart.append(pat_id)
        amount_of_patients_used_not_lennart += 1

unused_pats_not_Lennart = modifiedPatientsDataFrame[modifiedPatientsDataFrame.ID.isin(pats_not_used_not_Lennart)]

print(f"# of those not used for the dataset: {amount_of_patients_used_not_lennart}")
print("\ndue to the following reasons:")

unused_value_counts = unused_pats_not_Lennart.reason.value_counts()
print(unused_value_counts)
amount_of_patients_not_lennart_not_used_with_reasons = 0
for value in unused_value_counts:
    amount_of_patients_not_lennart_not_used_with_reasons += value

amount_other_reason_not_lennart = amount_of_patients_used_not_lennart - amount_of_patients_not_lennart_not_used_with_reasons
print(f"not correct amount of MRI sequences: {amount_other_reason_not_lennart}")


# of patients not selected by Lennart: 262
# of those not used for the dataset: 35

due to the following reasons:
reason
braintumor             2
CUP                    2
lymphoma               1
no brain metastases    1
Name: count, dtype: int64
not correct amount of MRI sequences: 29


#### patients selected by Lennart

In [113]:
# get all patients selected by Lennart
# compare with patients actually used
# list reasons why not used
# get funky

pats_by_Lennart = list(modifiedPatientsDataFrame[modifiedPatientsDataFrame["selected by"] == "Lennart"]["ID"])
pats_not_used_Lennart = []

print(f"# of patients selected by Lennart: {len(pats_by_Lennart)}")

amount_of_patients_used_lennart = 0

for pat_id in pats_by_Lennart:
    if pat_id not in list(dataset_patients["participant_id"]):
        pats_not_used_Lennart.append(pat_id)
        amount_of_patients_used_lennart += 1

unused_pats_Lennart = modifiedPatientsDataFrame[modifiedPatientsDataFrame.ID.isin(pats_not_used_Lennart)]

print(f"# of those not used for the dataset: {amount_of_patients_used_lennart}")
print("\ndue to the following reasons:")

unused_value_counts = unused_pats_Lennart.reason.value_counts()
print(unused_value_counts)
amount_of_patients_lennart_not_used_with_reasons = 0
for value in unused_value_counts:
    amount_of_patients_lennart_not_used_with_reasons += value

amount_other_reason_lennart = amount_of_patients_used_lennart - amount_of_patients_lennart_not_used_with_reasons

print()
print(f"not correct amount of MRI sequences: {amount_other_reason_lennart}")


# of patients selected by Lennart: 483
# of those not used for the dataset: 219

due to the following reasons:
reason
no brain metastases          58
brain tumor                  32
lack of data                 21
pre OP MRI missing           18
lymphoma                     17
histology unclear             8
CUP                           7
other tumor                   3
braintumor                    1
meningeosis carcinomatosa     1
Name: count, dtype: int64

not correct amount of MRI sequences: 53


acceptable reasons:\
no brain metastastes, brain tumor, other tumor, lymphoma, pre OP MRI missing, meningeosis carcinomatosa, CUP, missing MRI sequences, lack of data, histology unclear

## Data Analysis

To-do:
- sex
- age at MRI
- age at first diagnosis
- age at first diagnosis brainmet
- did any of the following occur during their therape: OP / Chemo / Radiatio / Immunotherapie
- height
- weight
- ECOG
- status of resection
- status of relapse

Try to display as such:

CATEGORY\
total n=XXX, mean m: XX (n=XXX) ± STD, mean f: XX (n=XXX) ± STD

In [209]:
dataset_pat_ids = list(dataset_patients["participant_id"])
excel_dataset_pats = modifiedPatientsDataFrame.copy()[modifiedPatientsDataFrame.ID.isin(dataset_pat_ids)]
#print(excel_dataset_pats)
excel_dataset_pats[excel_dataset_pats.duplicated(subset="ID")]

,on_server,"in_study (0 = no, 1 = yes, 2 = tbd, 3 = remarkable)",reason,comment,selected by,MRI_comments,ID,birthdate,sex (m/w),preop_MRICE_date,...,date_first_diagnosis_primary,date_first_diagnosis_brain_met,OP,Chemotherapie,Radiatio,Immuntherapie,Größe (cm),Gewicht (kg),ECOG,AZ Beschreibung


removed 2nd instance of `1349100` in excel sheet\
removed 1st instance of `1935938` in excel sheet\
removed 1st instance of `1699419` in excel sheet\
removed 2nd instance of `1876862` in excel sheet\
removed 1st instance of `1764802` in excel sheet\
removed 1st instance of `1456719` in excel sheet\
removed 1st instance of `2014467` in excel sheet\
removed 1st instance of `1792136` in excel sheet

In [197]:
# get list of patIDs of men and women in dataset
men_ids = list(excel_dataset_pats[excel_dataset_pats["sex (m/w)"] == "m"].ID)
women_ids = list(excel_dataset_pats[excel_dataset_pats["sex (m/w)"] == "w"].ID)

### sex

In [182]:
def percentage(part, whole):
  Percentage = float(part)/float(whole)
  return Percentage

pat_count = len(excel_dataset_pats)
for val, cnt in excel_dataset_pats["sex (m/w)"].value_counts().items():
    print("{:1s}: {:3d}/{:3d} ({:.2%})".format(val, cnt, pat_count, percentage(cnt, pat_count)))

m: 268/491 (54.58%)
w: 223/491 (45.42%)


### age at mri

In [117]:
ages_men = dataset_patients.age[dataset_patients.participant_id.isin(men_ids)]
mean_ages_men = np.mean(ages_men)
median_ages_men = np.median(ages_men)
std_ages_men = np.std(ages_men)

ages_women = dataset_patients.age[dataset_patients.participant_id.isin(women_ids)]
mean_ages_women = np.mean(ages_women)
median_ages_women = np.median(ages_women)
std_ages_women = np.std(ages_women)

print("{:^20}".format("ages men"))
print("{:12s}={:>6d}".format("total: n", len(ages_men)))
print("{:12s}={:>9.2f}".format("mean: m", mean_ages_men))
print("{:12s}={:>9.2f}".format("median: med", median_ages_men))
print("{:12s}={:>9.2f}".format("std", std_ages_men))
print()

print("{:^20}".format("ages women"))
print("{:12s}={:>6d}".format("total: n", len(ages_women)))
print("{:12s}={:>9.2f}".format("mean: m", mean_ages_women))
print("{:12s}={:>9.2f}".format("median: med", median_ages_women))
print("{:12s}={:>9.2f}".format("std", std_ages_women))

      ages men      
total: n    =   268
mean: m     =    63.69
median: med =    65.00
std         =    11.61

     ages women     
total: n    =   223
mean: m     =    60.78
median: med =    61.00
std         =    12.10


### primaries count

In [118]:
primaries_men = dataset_patients.primary[dataset_patients.participant_id.isin(men_ids)]
print("{:^44}".format("primaries men - detailed"))

for val, cnt in primaries_men.value_counts().items():
    primary_name = config.cancer_primaries_dict[val]
    print("{:38s}{:>6d}".format(primary_name, cnt))

          primaries men - detailed          
adenocarcinoma of the lung                67
melanoma                                  49
small cell lung cancer                    34
colorectal cancer                         15
NSCLC                                     14
lung cancer, exact histology unknown      13
squamous cell lung cancer                 12
prostate cancer                            8
transitional cell carcinoma                8
renal cell carcinoma                       8
lung cancer, other                         7
cancer of unknown primary                  5
esophageal cancer                          4
testicular cancer                          4
oral squamous cell carcinoma               3
laryngeal cancer                           2
osteosarcoma                               2
large cell lung cancer                     2
thyroid cancer                             1
breast cancer                              1
rhabdomyosarcoma                           1
liposarcom

In [119]:
# find patients with cancers of unknown primareis, shouldn't be in the dataset
# fuck
cup_patients = excel_dataset_pats[excel_dataset_pats.primary_coded_newv == "29"]
cup_patients

,on_server,"in_study (0 = no, 1 = yes, 2 = tbd, 3 = remarkable)",reason,comment,selected by,MRI_comments,ID,birthdate,sex (m/w),preop_MRICE_date,...,date_first_diagnosis_primary,date_first_diagnosis_brain_met,OP,Chemotherapie,Radiatio,Immuntherapie,Größe (cm),Gewicht (kg),ECOG,AZ Beschreibung
19,0,1,NaN,NaN,not Lennart,MRI images missing,1475524,1939-04-04,w,2006-07-28 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,0,1,NaN,NaN,not Lennart,MRI images missing,1434617,1950-10-18,m,2006-08-18 00:00:00,...,April 2003,ca. August 2005,1.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN
93,0,1,NaN,NaN,not Lennart,MRI images missing,1614295,1949-03-26,m,2011-03-18 00:00:00,...,März 2011,März 2011,1.0,1.0,1.0,0.0,176.0,75,NaN,NaN
96,0,1,NaN,NaN,not Lennart,MRI images missing,1575055,1968-06-08,m,2011-06-30 00:00:00,...,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN
109,1,1,NaN,NaN,not Lennart,NaN,1668799,1952-06-17,m,2012-06-28 00:00:00,...,September 2011,Juli 2012,1.0,0.0,1.0,0.0,180.0,70,1.0,NaN
119,0,1,NaN,NaN,not Lennart,MRI images missing,1641510,1937-04-27,m,2012-10-30 00:00:00,...,November 2012,November 2012,1.0,0.0,0.0,0.0,174.0,78,1.0,NaN
171,0,1,NaN,NaN,not Lennart,MRI images missing,1853095,1951-08-09,w,2016-06-13 00:00:00,...,Mai 2007 (Vulva-Ca) oder August 2014 (Mamma-Ca),Juni 2016,1.0,0.0,0.0,1.0,172.0,64,0.0,NaN


In [120]:
primaries_women = dataset_patients.primary[dataset_patients.participant_id.isin(women_ids)]
print("{:^44}".format("primaries women - detailed"))

for val, cnt in primaries_women.value_counts().items():
    primary_name = config.cancer_primaries_dict[val]
    print("{:38s}{:>6d}".format(primary_name, cnt))

         primaries women - detailed         
adenocarcinoma of the lung                53
breast cancer                             40
melanoma                                  28
small cell lung cancer                    24
NSCLC                                     16
colorectal cancer                          9
squamous cell lung cancer                  8
renal cell carcinoma                       5
lung cancer, exact histology unknown       4
endometrial cancer                         4
lung cancer, other                         4
ovarian cell carcinoma                     4
sarcoma, other                             3
cancer of unknown primary                  2
salivary gland cancer                      2
cervical cancer                            2
gastric cancer                             2
leiomyosarcoma                             2
liposarcoma                                1
laryngeal cancer                           1
oral squamous cell carcinoma               1
synovial s

### age at first diagnosis tumor

In [212]:
# get values of first diagnosis
# find difficult values
# decide wether to ignore or not
# compute age
# print age

months = {
    "Januar ":"01.01.",
    "Februar ":"01.02.",
    "März ":"01.03.",
    "April ":"01.04.",
    "Mai ":"01.05.",
    "Juni ":"01.06.",
    "Juli ":"01.07.",
    "August ":"01.08.",
    "September ":"01.09.",
    "Oktober ":"01.10.",
    "November ":"01.11.",
    "Dezember ":"01.12.",
    "Jan ":"01.01.",
    "Feb ":"01.02.",
    "Apr ":"01.03.",
    "Jun ":"01.06.",
    "Jul ":"01.07.",
    "Aug ":"01.08.",
    "Sept ":"01.09.",
    "Sep ":"01.09.",
    "Okt ":"01.10.",
    "Nov ":"01.11.",
    "Dez ":"01.12."
}

excel_dataset_pats.date_first_diagnosis_primary = excel_dataset_pats.date_first_diagnosis_primary.astype("str")

for month in months.keys():
    excel_dataset_pats.date_first_diagnosis_primary = excel_dataset_pats.date_first_diagnosis_primary.str.replace(month, months[month])


print(excel_dataset_pats.date_first_diagnosis_primary)

#replace months with numbers
# excel_dataset_pats.date_first_diagnosis_primary = excel_dataset_pats.date_first_diagnosis_primary.str.replace("Januar", "01")

#print(excel_dataset_pats.date_first_diagnosis_primary)

excel_dataset_pats.date_first_diagnosis_primary = pd.to_datetime(excel_dataset_pats.date_first_diagnosis_primary, errors="raise", format="mixed")
excel_dataset_pats.birthdate = pd.to_datetime(excel_dataset_pats.birthdate, errors="raise", format="mixed")


0      2003-09-01 00:00:00
1      2005-04-01 00:00:00
2      2004-09-01 00:00:00
4      2005-05-01 00:00:00
5      2003-01-01 00:00:00
              ...         
735    2023-10-01 00:00:00
736    2021-01-01 00:00:00
740    2023-10-01 00:00:00
741    2023-02-01 00:00:00
744    2023-10-01 00:00:00
Name: date_first_diagnosis_primary, Length: 491, dtype: object


#### men

In [213]:
men_age_first_diagnosis_dict = {}
for index, row in excel_dataset_pats[excel_dataset_pats.ID.isin(men_ids)].iterrows():
    id = row["ID"]

    # calculate the age of the patient when first diagnosed with tumor
    if not pd.isnull(row.date_first_diagnosis_primary):
        age = relativedelta(row.date_first_diagnosis_primary, row.birthdate).years
        men_age_first_diagnosis_dict[id] = age
    else:
        #print(id)
        pass


men_age_first_diagnosis_dict = list(men_age_first_diagnosis_dict.values())

mean_ages_first_diagnosis_men = np.mean(men_age_first_diagnosis_dict)
median_ages_first_diagnosis_men = np.median(men_age_first_diagnosis_dict)
std_ages_first_diagnosis_men = np.std(men_age_first_diagnosis_dict)

print("{:^50}".format("ages of men first diagnosed with the tumor"))
print("{:12s}={:>6d}".format("total: n", len(men_age_first_diagnosis_dict)))
print("{:12s}={:>9.2f}".format("mean: m", mean_ages_first_diagnosis_men))
print("{:12s}={:>9.2f}".format("median: med", median_ages_first_diagnosis_men))
print("{:12s}={:>9.2f}".format("std", std_ages_first_diagnosis_men))


    ages of men first diagnosed with the tumor    
total: n    =   261
mean: m     =    60.89
median: med =    63.00
std         =    14.71


#### women

In [214]:
women_age_first_diagnosis_dict = {}
for index, row in excel_dataset_pats[excel_dataset_pats.ID.isin(women_ids)].iterrows():
    id = row["ID"]

    # calculate the age of the patient when first diagnosed with tumor
    if not pd.isnull(row.date_first_diagnosis_primary):
        age = relativedelta(row.date_first_diagnosis_primary, row.birthdate).years
        women_age_first_diagnosis_dict[id] = age


women_age_first_diagnosis_dict = list(women_age_first_diagnosis_dict.values())

mean_ages_first_diagnosis_women = np.mean(women_age_first_diagnosis_dict)
median_ages_first_diagnosis_women = np.median(women_age_first_diagnosis_dict)
std_ages_first_diagnosis_women = np.std(women_age_first_diagnosis_dict)

print("{:^20}".format("ages women"))
print("{:12s}={:>6d}".format("total: n", len(women_age_first_diagnosis_dict)))
print("{:12s}={:>9.2f}".format("mean: m", mean_ages_first_diagnosis_women))
print("{:12s}={:>9.2f}".format("median: med", median_ages_first_diagnosis_women))
print("{:12s}={:>9.2f}".format("std", std_ages_first_diagnosis_women))

     ages women     
total: n    =   216
mean: m     =    55.56
median: med =    58.00
std         =    19.49
